# Cross Validation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier


In [ ]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

df = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
df.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [ ]:
X = df[['preco', 'idade_do_modelo', 'km_por_ano']]
y = df['vendido']

In [ ]:
SEED = 158020
np.random.seed(SEED)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, stratify=y)

X_train.shape, X_test.shape

((7500, 3), (2500, 3))

In [ ]:
# baseline
dummy_stratified = DummyClassifier()
dummy_stratified.fit(X_train, y_train)
accuracy_dummy = dummy_stratified.score(X_test, y_test)
accuracy_dummy

/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


0.5096

In [ ]:
clf = DecisionTreeClassifier(max_depth=2)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
accuracy_score(y_test, predictions) 

0.7192

Cliente quer acurácia mínima de 73%

E se usar outro seed?

In [ ]:
SEED = 301
np.random.seed(SEED)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, stratify=y)

clf = DecisionTreeClassifier(max_depth=2)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
accuracy_score(y_test, predictions) 

# mudou o seed, a acurácia aumentou muito - efeitos da aleatoriedade

0.7728

# Cross Validation

In [ ]:
SEED = 301
np.random.seed(SEED)

model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, X,y, cv=3, return_train_score=False)
media = results['test_score'].mean()
desvio = results['test_score'].std()

print("Accuracy com cross validation, 3 = [%.2f, %.2f]" % ((media -2 *desvio)*100, (media + 2*desvio )*100))

# ja estamos menossucetíveis a escplha da semente aleatória, mudando o SEED os resultados se mantém
# cross validate nao receve o SEED como parametro (que coisa não)
# mas quem garante o valor 3 para cv???

Accuracy com cross validation, 3 = [74.99, 76.57]


In [ ]:
SEED = 301
np.random.seed(SEED)

model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, X,y, cv=10, return_train_score=False)
media = results['test_score'].mean()
desvio = results['test_score'].std()

print("Accuracy com cross validation, 10 = [%.2f, %.2f]" % ((media -2 *desvio)*100, (media + 2*desvio )*100))

Accuracy com cross validation, 10 = [74.24, 77.32]


In [ ]:
SEED = 301
np.random.seed(SEED)

model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, X,y, cv=3, return_train_score=False)
media = results['test_score'].mean()
desvio = results['test_score'].std()

print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media -2 *desvio)*100, (media + 2*desvio )*100))

# que numero usar para cross validation?
# pelos papers entre 5 e 10 - portano, vamos manter 05

Accuracy com cross validation, 5 = [74.99, 76.57]


# Cross validation dividindo o dataset de maneira aleatória de verdade!

In [ ]:
def print_results(results):
  media = results['test_score'].mean()
  desvio = results['test_score'].std()
  print("Accuracy médio = %.2f" % (media*100))
  print("Accuracy intervalo = [%.2f, %.2f]" % ((media - 2*desvio)*100, (media +2*desvio)*100))  

In [ ]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, X, y, cv = cv, return_train_score=False)
print_results(results)

Accuracy médio = 75.78
Accuracy intervalo = [74.37, 77.19]


In [ ]:
# com shuffle no KFold
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, X, y, cv = cv, return_train_score=False)
print_results(results)

Accuracy médio = 75.76
Accuracy intervalo = [73.26, 78.26]


# simular situação de azar 

classes desbalanceadas

In [ ]:
df_azar = df.sort_values('vendido', ascending=True)
X_azar = df_azar[['preco', 'idade_do_modelo', 'km_por_ano']]
y_azar = df_azar['vendido']

In [ ]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, X_azar, y_azar, cv = cv, return_train_score=False)
print_results(results)

# diferença no resultado

Accuracy médio = 57.84
Accuracy intervalo = [34.29, 81.39]


In [ ]:
# com shuffle - ja ajuda bastante a balancear de novo

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, X_azar, y_azar, cv = cv, return_train_score=False)
print_results(results)

Accuracy médio = 75.78
Accuracy intervalo = [72.30, 79.26]


In [ ]:
# StratifiedKFold (quando as classes estão muito desbalanceadas)
SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits=10, shuffle=True)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, X_azar, y_azar, cv = cv, return_train_score=False)
print_results(results)

Accuracy médio = 75.78
Accuracy intervalo = [73.55, 78.01]


# 

In [ ]:
np.random.seed(SEED)
df['modelo'] = df.idade_do_modelo + np.random.randint(-2,3, size=df.shape[0])
df.modelo = df.modelo + abs(df.modelo.min()) + 1
df.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [ ]:
df.modelo.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1])

In [ ]:
df.modelo.value_counts()

20    901
19    798
18    771
21    723
17    709
16    668
14    621
22    575
15    573
13    557
12    511
11    401
10    371
23    370
9     336
8     278
7     206
24    199
6     181
5     108
4      76
3      44
2      17
1       6
Name: modelo, dtype: int64

# Testar cross validation com GroupKFold

In [ ]:
# testar estimador com grupos (especificar os grupos)
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, X_azar, y_azar, cv = cv, groups=df.modelo,return_train_score=False)
print_results(results)

Accuracy médio = 75.78
Accuracy intervalo = [73.67, 77.90]


In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
train_x_scaled = scaler.transform(X_train)
test_x_scaled = scaler.transform(X_test)

In [ ]:
SEED = 301
np.random.seed(SEED)

model = SVC()
model.fit(train_x_scaled, y_train)
predictions = model.predict(test_x_scaled)

acuracia = accuracy_score(y_test, predictions)*100
print("Accuracy %.2f%%" % acuracia)

Accuracy 77.12%


In [ ]:
# testar com cross validation
SEED = 301
np.random.seed(SEED)

# dados nao escalados
cv = GroupKFold(n_splits=10)
model = SVC()
results = cross_validate(model, X_azar, y_azar, cv = cv, groups=df.modelo,return_train_score=False)
print_results(results)

Accuracy médio = 77.27
Accuracy intervalo = [74.35, 80.20]


In [ ]:
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao',scaler), ('estimador',modelo)])

cv = GroupKFold(n_splits = 10)
results = cross_validate(pipeline, X_azar, y_azar, cv = cv, groups = df.modelo, return_train_score=False)
print_results(results)

Accuracy médio = 76.68
Accuracy intervalo = [74.28, 79.08]
